In [1]:
from project1 import BatchNormalization,DarknetConv,DarknetResidual,DarknetBlock,Darknet,YoloConv,YoloOutput,yolo_boxes,yolo_nms,YoloV3,load_darknet_weights,transform_images

In [2]:
import cv2
from PIL import Image
import numpy as np
import tensorflow as tf
import urllib3

In [3]:
import time

In [4]:
YOLOV3_LAYER_LIST = [
    'yolo_darknet',
    'yolo_conv_0',
    'yolo_output_0',
    'yolo_conv_1',
    'yolo_output_1',
    'yolo_conv_2',
    'yolo_output_2',
]

In [5]:
yolo_anchors = np.array([(10, 13), (16, 30), (33, 23), (30, 61), (62, 45),
                         (59, 119), (116, 90), (156, 198), (373, 326)],
                        np.float32) / 416
yolo_anchor_masks = np.array([[6, 7, 8], [3, 4, 5], [0, 1, 2]])

In [6]:
class_names = [c.strip() for c in open('coco.names').readlines()]
n_classes=len(class_names)

In [7]:
yolo = YoloV3(classes=n_classes)

In [8]:
load_darknet_weights(yolo,'yolov3.weights')

In [9]:
'''
def decode_image(img_path):
    #img = cv2.imread(img_path)
    img = tf.image.decode_image(open(img_path, 'rb').read(), channels=3)
    img = tf.expand_dims(img, 0)
    img_test = transform_images(img, 416)
    return (img_test)
'''
def decode_image(img):
    img = tf.convert_to_tensor(img)
    img = tf.expand_dims(img, 0)
    img_test = transform_images(img, 416)
    return (img_test)

In [10]:
def calculate(outputs, class_names):
    vehicle = 0
    pedestrian = 0
    boxes, objectness, classes, nums = outputs
    boxes, objectness, classes, nums = boxes[0], objectness[0], classes[0], nums[0]
    #print(classes)

    for i in range(nums):
        if (class_names[int(classes.numpy()[i])]) == "person":
            pedestrian = pedestrian+1
        elif (class_names[int(classes.numpy()[i])]) == "car" or (class_names[int(classes.numpy()[i])]) == "truck" or (class_names[int(classes.numpy()[i])]) == "bus":
            vehicle = vehicle+1
    return(vehicle,pedestrian)


In [11]:
def draw_outputs(img, outputs, class_names):
    boxes, objectness, classes, nums = outputs
    boxes, objectness, classes, nums = boxes[0], objectness[0], classes[0], nums[0]
    wh = np.flip(img.shape[0:2])
    for i in range(nums):
        if (class_names[int(classes.numpy()[i])]) == "person":
            x1y1 = tuple((np.array(boxes[i][0:2]) * wh).astype(np.int32))
            x2y2 = tuple((np.array(boxes[i][2:4]) * wh).astype(np.int32))
            img = cv2.rectangle(img, x1y1, x2y2, (255, 0, 0), 2)
            img = cv2.putText(img, '{} {:.4f}'.format(class_names[int(classes[i])], objectness[i]),
                              x1y1, cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2)
            
        elif (class_names[int(classes.numpy()[i])]) == "car" or (class_names[int(classes.numpy()[i])]) == "truck" or (class_names[int(classes.numpy()[i])]) == "bus":
            x1y1 = tuple((np.array(boxes[i][0:2]) * wh).astype(np.int32))
            x2y2 = tuple((np.array(boxes[i][2:4]) * wh).astype(np.int32))
            img = cv2.rectangle(img, x1y1, x2y2, (255, 0, 0), 2)
            img = cv2.putText(img, '{} {:.4f}'.format(class_names[int(classes[i])], objectness[i]),
                              x1y1, cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2)
    return img

In [14]:

def read_cam():
    import requests
    myAPI = "M7VF4BS94JYVEC0L"
    baseURL = 'https://api.thingspeak.com/update'
    #cap = cv2.VideoCapture('Cars moving on road Stock Footage - Free Download.mp4')
    cap = cv2.VideoCapture(0)
    
    #cap = cv2.VideoCapture('Busy traffic in Kolkata - West Bengal.mp4')
    frame_count = 0
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        frame_count = frame_count+1
        if ret == True:
            #cv2.imwrite("test.jpg",frame)
            #img_path = "test.jpg"
            boxes, scores, classes, nums = yolo(decode_image(frame))
            img= draw_outputs(frame, (boxes, scores, classes, nums), class_names)
            v,p = calculate((boxes, scores, classes, nums),class_names)
            print(v," ",p)
            payload = {'api_key': myAPI, 'field1': str(v),'field2': str(p)}
            a = requests.post(baseURL,params = payload)
            cv2.imshow("output",img)
            #cv2.imwrite('C://Users/DEBANJAN GHOSH/Downloads/video/image'+str(frame_count)+'.jpeg',img)
        if cv2.waitKey(1) == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [15]:
read_cam()

0   6
0   4
0   6
0   7
0   7
0   5
0   7
1   3
1   6
1   6
1   3
1   3
0   6
0   4
0   6
0   5
1   4
1   7
1   7
1   5
1   6
1   4
0   5
1   3
1   2
1   1
0   4
0   3
0   4
0   4
0   4
0   6
0   8
0   9
0   6
0   6
0   6
0   7
0   9
0   7
0   7
0   6
0   6
0   8
0   7
0   7
0   5
0   6
0   5
0   6
0   7
0   5
0   6
0   5
0   4
0   5
0   5
0   4
0   3
0   4
0   5
0   3
0   6
0   4
0   6
0   3
0   4
0   2
0   3
0   6
0   5
0   7
0   5
0   7
0   4
0   4
0   4
0   5
0   6
0   7
0   6
0   6
0   6
0   7
0   4
0   4
0   3
0   4
0   5
0   6
0   7
0   6
0   6
0   6
0   4
0   3
0   4
0   3
0   5
0   7
0   4
0   7
0   6
0   4
0   6
0   4
0   7
0   5
0   6
0   8
0   5
0   8
0   9
0   6
0   5
0   8
0   7
0   5
0   6
0   8
0   8
0   6
0   5
0   5
0   7


In [18]:
def pic():
    img = cv2.imread('IMG_20191115_154336.jpg')
    boxes, scores, classes, nums = yolo(decode_image("IMG_20191115_154336.jpg"))
    img= draw_outputs(img, (boxes, scores, classes, nums), class_names)
    cv2.imshow("output",img)
    if cv2.waitKey(1)== ord('q'):
        cv2.destroyAllWindows()
        return


In [ ]:
pic()

In [ ]:
def video